# Import modules and init in gee


In [2]:
import copy
import os
from typing import Dict, Any, List, Union, Sequence, Tuple
from collections import defaultdict

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from scipy.stats.mstats import theilslopes
from hampel import hampel


plt.rcParams['axes.edgecolor'] = 'blue'
plt.rcParams['figure.figsize'] = [14, 7]

# Implementation


### caclulation statistics

In [ ]:
def mae_log(in_situ: Sequence[float], pred: Sequence[float]) -> float:
    """Вычисляет среднюю абсолютную ошибку в логарифмическом масштабе (MAE_log).
    
    Args:
        in_situ: Истинные значения (измеренные данные)
        pred: Предсказанные значения
        
    Returns:
        MAE_log: Значение метрики, рассчитанной как среднее абсолютное отклонение логарифмов
    """
    
    res = 0
    y_pred_pos = np.maximum(np.array(pred), 0.01)
    for i, ai in enumerate(in_situ):
        res += abs(np.log10(y_pred_pos[i]) - np.log10(ai))
    res /= len(in_situ)
    return 10 ** res


def bias_log(in_situ: Sequence[float], pred: Sequence[float]) -> float:
    """Вычисляет систематическую ошибку в логарифмическом масштабе (BIAS_log).
    
    Args:
        in_situ: Истинные значения
        pred: Предсказанные значения
        
    Returns:
        BIAS_log: Среднее значение разности логарифмов предсказаний и истинных значений
    """
    
    res = 0
    y_pred_pos = np.maximum(np.array(pred), 0.01)
    for i, ai in enumerate(in_situ):
        res += np.log10(y_pred_pos[i]) - np.log10(ai)
    res /= len(in_situ)
    return 10 ** res


def r2_cov(y: Sequence[float], y_pred: Sequence[float]) -> float:
    """Вычисляет коэффициент детерминации R² через ковариацию.
    
    Args:
        y: Истинные значения
        y_pred: Предсказанные значения
        
    Returns:
        R²: Значение метрики от 0 до 1, где 1 - идеальное совпадение
    """
    
    y = np.array(y)
    y_pred = np.array(y_pred)
    y_bar = y.mean()
    y_pred_bar = y_pred.mean()
    return np.pow(np.sum((y - y_bar) * (y_pred - y_pred_bar)) /
                  np.sqrt(np.sum(np.pow(y - y_bar, 2)) * np.sum(np.pow(y_pred - y_pred_bar, 2))), 2)


def bias(in_situ: Sequence[float], pred: Sequence[float]) -> float:
    """Вычисляет систематическую ошибку (BIAS).
    
    Args:
        in_situ: Истинные значения
        pred: Предсказанные значения
        
    Returns:
        BIAS: Среднее значение разности между предсказаниями и истинными значениями
    """
    
    res = 0
    for i, ai in enumerate(in_situ):
        res += (pred[i] - ai)
    return res / len(in_situ)


def sen_slope(list_in_situ: Sequence[float], list_calc: Sequence[float]) -> float:
    """Вычисляет наклон по Тейлу-Сену между двумя наборами данных.
    
    Args:
        list_in_situ: Первый набор данных
        list_calc: Второй набор данных
        
    Returns:
        slope: Наклон регрессии Тейла-Сена
    """
    
    return theilslopes(list_in_situ, list_calc).slope


def std(in_situ: Sequence[float], pred: Sequence[float]) -> float:
    """Вычисляет стандартное отклонение разности предсказаний и истинных значений.
    
    Args:
        in_situ: Истинные значения
        pred: Предсказанные значения
        
    Returns:
        std: Стандартное отклонение разности
    """
    
    return np.std(np.array(pred) - np.array(in_situ))


def calculate_statistics(list_in_situ: Sequence[float], list_calc: Sequence[float]) -> List[float]:
    """Рассчитывает набор статистических метрик для оценки качества предсказаний.
    
    Args:
        list_in_situ: Истинные значения
        list_calc: Предсказанные значения
        
    Returns:
        Список метрик в следующем порядке:
        [R², BIAS_log, BIAS, MAE_log, MAE, RMSE, STD, Sen_slope, MAPE]
        где:
        - R²: Коэффициент детерминации
        - BIAS_log: Систематическая ошибка в логарифмическом масштабе
        - BIAS: Среднее отклонение
        - MAE_log: Средняя абсолютная ошибка в логарифмическом масштабе
        - MAE: Средняя абсолютная ошибка
        - RMSE: Корень из среднеквадратичной ошибки
        - STD: Стандартное отклонение разности
        - Sen_slope: Наклон по Тейлу-Сену
        - MAPE: Средняя абсолютная процентная ошибка (%)
    """
    
    arr = [r2_cov(list_in_situ, list_calc), 
           bias_log(list_in_situ, list_calc),
           bias(list_in_situ, list_calc), 
           mae_log(list_in_situ, list_calc), 
           mean_absolute_error(list_in_situ, list_calc),
           np.sqrt(mean_squared_error(list_in_situ, list_calc)), 
           std(list_in_situ, list_calc),
           sen_slope(list_calc, list_in_situ), 
           mean_absolute_percentage_error(list_in_situ, list_calc) * 100]
    return list(map(lambda x: round(x, 3), arr))

### draw and estimated chl

In [ ]:
def draw_graphic(list1: Sequence[float], list2: Sequence[float], name_algo: str, save_folder: str = None) -> float:
    """
    Строит регрессионный график с линейной регрессией и оценочную функцию Тейла-Сена.
    
    Args:
        list1: Истинные значения (измеренные данные)
        list2: Предсказанные значения
        name_algo: Название алгоритма для подписи графика
        save_folder: Папка для сохранения графика (по умолчанию None - отображает график)
    
    Returns:
        float: Стандартное отклонение разности между истинными и предсказанными значениями
        
    Raises:
        ValueError: Если длины входных последовательностей не совпадают
    """
    
    if len(list1) != len(list2):
        raise ValueError("Длины входных последовательностей должны совпадать")
    
    x = np.array(copy.deepcopy(list1)).reshape((-1, 1))
    y1 = np.array(copy.deepcopy(list2))
    
    model = LinearRegression().fit(x, y1)
    x_dots = np.linspace(x.min(), x.max(), 100).reshape((-1, 1))
    y_pred = model.predict(x_dots)
    
    plt.figure(figsize=(10, 8))
    plt.scatter(x, y1, c=[(0.7, 0.2, 0.9)], s=150,
                edgecolor='black', linewidth=1.5, alpha=0.9, 
                marker='o', zorder=3, label='Измеренные данные')
    
    plt.plot(x_dots, y_pred, 'b', label='Линейная регрессия')
    
    st = theilslopes(list2, list1)
    plt.plot(x_dots, st.slope * x_dots + st.intercept, color='green', label='Оценочная функция Тейла-Сена')
    
    plt.plot(x_dots, x_dots, '--', color='black', label='1:1 линия')
    
    plt.xlabel(f"in_situ, мкг/л\nN = {len(list1)}, Наклон Тейла-Сена = {st.slope:.3f}", fontsize=16, labelpad=10)
    plt.ylabel(f"{name_algo}, мкг/л", fontsize=16, labelpad=10)
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    
    if save_folder:
        os.makedirs(f"graphics/{save_folder}", exist_ok=True)
        filename = f"{name_algo}.png"
        save_path = os.path.join(f"graphics/{save_folder}", filename)
        plt.savefig(save_path, bbox_inches="tight")
        plt.close()
    else:
        plt.show()
    
    return round(np.std(np.array(list2) - np.array(list1)), 3)


def calculate_statistics_and_draw_graphic(df_in_situ: pd.DataFrame, **kwargs: Dict[str, Dict[Any, float]]) -> Dict[str, List[Any]]:
    """
    Рассчитывает статистики и строит графики для множества алгоритмов.
    
    Args:
        df_in_situ: DataFrame с истинными значениями (столбец 'CHL')
        **kwargs: Словари с результатами алгоритмов в формате {имя_алгоритма: {ключ: значение}}
    
    Returns:
        Dict[str, List[Any]]: Словарь со статистиками по каждому алгоритму, содержащий:
            - [R², BIAS_log, BIAS, MAE_log, MAE, RMSE, STD, Sen_slope, MAPE]
            - Количество точек
            - Фильтрованные истинные значения
            - Фильтрованные предсказанные значения
    """
    
    dict_in_situ = {key: value for key, value in zip(df_in_situ.index.to_list(), df_in_situ['CHL'].to_list())}
    statistics = dict()
    
    for algo_name, algo_dict in kwargs.items():
        tmp_copy_calc = copy.deepcopy(algo_dict)
        
        for k, v in tmp_copy_calc.items():
            if v is None or v <= 0 or np.isnan(v) or abs(dict_in_situ[k] - tmp_copy_calc[k]) > 100:
                del algo_dict[k]
        
        tmp_copy_in_situ = copy.deepcopy(dict_in_situ)
        for k in list(tmp_copy_in_situ.keys()):
            if k not in algo_dict:
                del tmp_copy_in_situ[k]
        
        in_situ_values = list(tmp_copy_in_situ.values())
        calc_chl = list(algo_dict.values())
        
        if len(calc_chl) > 3:
            statistics[algo_name] = [*calculate_statistics(in_situ_values, calc_chl), 
                                   len(calc_chl), 
                                   tmp_copy_in_situ, 
                                   algo_dict]
            
            draw_graphic(in_situ_values, calc_chl, algo_name, save_folder="regional")
        else:
            print(f"Недостаточно данных для алгоритма {algo_name} (количество точек = {len(calc_chl)})")
    
    return statistics

### formulae in articles

#### Beck

In [ ]:
def beck(values: dict):
    ndci = (values['B5'] - values['B4']) / (values['B5'] + values['B4']) 
    flh_violet = values['B3'] - (values['B4'] + ((665 - 560.5) / (665 - 490.5) * (values['B2'] - values['B4'])))
    _2bda = values['B5'] / values['B4']
    _3bda = ((1 / values['B4']) - (1 / values['B5'])) * values['B8A']
    
    ndci_chl = None if (_ndci := 0.388 * ndci - 18.844) < 0 else _ndci
    flh_violet_chl = None if (_flh_violet_chl := -0.033 * flh_violet + 53.064) < 0 else _flh_violet_chl
    _2bda_chl = None if (__2bda_chl := _2bda * 86.148 - 51.94) < 0 else __2bda_chl
    _3bda_chl = None if (__3bda_chl := 156.286 * _3bda + 35.982) < 0 else __3bda_chl
    return ndci_chl, flh_violet_chl, _2bda_chl, _3bda_chl

#### Molkov

In [ ]:
def molkov_linear(x, a, b):
    a = np.array(a)
    b = np.array(b)
    return None if x is None or (_ := a * x + b) < 0 else _
    
    
def molkov_poly(x, a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    return None if x is None or (_ := a * np.pow(x, 2) + b * x + c) < 0 else _ 


def molkov_exp(x, a, b):
    a = np.array(a)
    b = np.array(b)
    return None if x is None else a * np.exp(b * x)
    
    
def molkov_power1(x, a, b):
    a = np.array(a)
    b = np.array(b)
    return None if x is None else a * np.pow(x, b)
    

def molkov_power2(x, a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    return None if (x is None) or np.any(a * x + b < 0) else np.pow((a * x + b), c)
        

def molkov(values: dict) -> tuple:
    """For each values of bands calculate indecies, which looks like lists [lin, poly, exp, p1, p2]

    Returns:
        tuple of lists: dim = 8: ndci_list, _2bda_list, _3bda_list, ph_list, \n
        _2b_linear_result, _2b_poly_result, ndci_linear_result, ndci_poly_result
    """
    
    ndci = (values['B5'] - values['B4']) / (values['B5'] + values['B4'])
    _2bda = values['B5'] / values['B4']
    _3bda = ((1 / values['B4']) - (1 / values['B5'])) * values['B6']
    ph = values['B5'] - (values['B6'] + values['B4']) / 2
    
    ndci_list, _2bda_list, _3bda_list, ph_list = np.zeros(5, dtype=np.float64), np.zeros(5, dtype=np.float64), \
        np.zeros(5, dtype=np.float64), np.zeros(5, dtype=np.float64)
    
    ndci_list[0], _2bda_list[0], _3bda_list[0], ph_list[0] = molkov_linear(ndci, 90.101, 13.75), \
        molkov_linear(_2bda, 24.463, -8.356), molkov_linear(_3bda, 35.812, 19.24), molkov_linear(ph, 2340.123, 9.185)
    
    ndci_list[1], _2bda_list[1], _3bda_list[1], ph_list[1] = molkov_poly(ndci, 88.757, 52.715, 15.06), \
        molkov_poly(_2bda, -2.155, 33.034, -15.59), molkov_poly(_3bda, -10.227, 52.867, 16.91), molkov_poly(ph, -17567.0, 2814.313, 7.11)
    
    ndci_list[2], _2bda_list[2], _3bda_list[2], ph_list[2] = molkov_exp(ndci, 16.365, 2.754), \
        molkov_exp(_2bda, 12.546, 0.526), molkov_exp(_3bda, 25.114, 0.554), molkov_exp(ph, 17.808, 52.313)
    
    ndci_list[3], _2bda_list[3], _3bda_list[3], ph_list[3] = None, molkov_power1(_2bda, 17.346, 1.198), \
        None, molkov_power1(ph, 890.568, 0.708)
    
    ndci_list[4], _2bda_list[4], _3bda_list[4], ph_list[4] = molkov_power2(ndci, 6.251, 3.359, 2.217), \
        molkov_power2(_2bda, 96.806, -61.71, 0.764), molkov_power2(_3bda, 4373.068, 296.6, 0.479), molkov_power2(ph, 7834.173, 8.051, 0.789)
    
    _2b_linear_result = molkov_linear(_2bda, 64.536, -57.8)
    _2b_poly_result = molkov_poly(_2bda, -73.669, 252.808, -176.68)
    ndci_linear_result = molkov_linear(ndci, 167.293, 4.756)
    ndci_poly_result = molkov_poly(ndci, -300.26, 235.556, 1.586)
    return ndci_list, _2bda_list, _3bda_list, ph_list, \
        _2b_linear_result, _2b_poly_result, ndci_linear_result, ndci_poly_result


def convert_molkov_dict(index_dict: dict) -> tuple:
    """For each pair flatten lists to dict: id: [0, 1, 2, ...] -> dict(id: 0), dict(id: 1), ...

    Returns:
        tuple of dicts
    """
    _0, _1, _2, _3, _4 = dict(), dict(), dict(), dict(), dict()
    for k, v in index_dict.items():
        _0[k] = v[0]
        _1[k] = v[1]
        _2[k] = v[2]
        _3[k] = v[3]
        _4[k] = v[4]
    return _0, _1, _2, _3, _4

#### Li

In [ ]:
def li_candidate13(x, a0, a1, a2, a3, a4):
    """
    Args:
        x: np.log10(values['B2'] / values['B3'])
    """
    return np.pow(10, a0 + a1 * x + a2 * np.pow(x, 2) + a3 * np.pow(x, 3) + a4 * np.pow(x, 4))


def li_candidate(values: dict):
    """
    ndci = None if values['B5'] < values['B4'] else (values['B5'] - values['B4']) / (values['B5'] + values['B4']) 
    _2bda1 = values['B6'] / values['B5']
    _2bda2 = values['B5'] / values['B4']
    _3bda = None if values['B5'] < values['B4'] else ((1 / values['B4']) - (1 / values['B5'])) * values['B6']
    """
    
    chl11 = None if (__chl11 := 136.3 * (values['B6'] / values['B4']) - 16.2) < 0 else __chl11
    
    chl12 = None if (__chl12 := 25.28 * (_chl12 := values['B5'] / values['B4'])**2 + 14.85 * _chl12 - 15.18) < 0 else __chl12
    
    chl13 = np.pow(10, 0.2389 - 1.9369 * (_chl13 := np.log10(values['B2'] / values['B3'])) +
                   1.7627 * _chl13**2 - 3.0777 * _chl13**3 - 0.1054 * _chl13**4)
    
    
    chl211 = None if \
        (__chl211 := 117.42 * ((1 / values['B4']) - (1 / values['B5'])) * values['B6'] + 23.09) < 0 else __chl211
        
    chl212 = None if \
        (__chl212 := 232.329 * ((1 / values['B4']) - (1 / values['B5'])) * values['B6'] + 23.174) < 0 else __chl212
        
    chl213 = None if \
        (__chl213 := 315.50 * (_chl213 := ((1 / values['B4'] - 1 / values['B5']) * values['B6']))**2 +
                      215.95 * _chl213 + 25.66) < 0 else __chl213
        
    chl22 = None if values['B6'] == values['B5'] or (__chl22 := \
        161.24 * (((1 / values['B4']) - (1 / values['B5'])) / ((1 / values['B6']) - (1 / values['B5']))) + 28.04) < 0 else __chl22
    return chl11, chl12, chl13, chl211, chl212, chl213, chl22

#### Makwinja

In [ ]:
def makwinja(values: dict):
    ndci = (values['B5'] - values['B4']) / (values['B5'] + values['B4'])
    return None if (_ndci := 431.98 * ndci**2 + 104 * ndci + 9.547) < 0 else _ndci

#### Boldanova

In [ ]:
def boldanova(values: dict):
    boldanova_b3 = None if (_boldanova_b3 := 3635.4 * (values['B3'])**2 - 185.7 * values['B3'] + 3.5) < 0 else _boldanova_b3
    return boldanova_b3

#### Karimi

In [ ]:
def karimi(values: dict):
    _2bda = None if values['B4'] == 0 else \
        np.exp(3.4 * (__2bda := values['B5'] / values['B4'])**2 - 6.6 * __2bda + 4.9)
    ndci = np.exp(5.83 * (_ndci := (values['B5'] - values['B4']) / (values['B5'] + values['B4']))**2 + 0.075 * _ndci + 1.72)
    _3bda = np.exp(2.6 * (__3bda := ((1 / values['B4']) - (1 / values['B5'])) * values['B8A'])**2 - 0.2 * __3bda + 1.74)
    return _2bda, ndci, _3bda

#### O'Reilly

In [ ]:
def oc3_msi(x, a0, a1, a2, a3, a4):
    return np.pow(10, a0 + a1 * x + a2 * np.pow(x, 2) + a3 * np.pow(x, 3) + a4 * np.pow(x, 4))


def oreilly(values: dict):
    oc3 = np.pow(10., 0.30963 - \
        2.40052 * (_oc3 := np.log10(max(values['B1'], values['B2']) / values['B3'])) + \
            1.28932 * _oc3**2 + 0.52802 * _oc3**3 - 1.33825 * _oc3**4)
    return oc3

### main

In [ ]:
def main(dict_stats: Dict[str, Dict[str, float]], df: pd.DataFrame, article: str) -> Dict[str, List[Union[float, int, Dict, Any]]]:
    """
    Обрабатывает данные по заданному алгоритму, строит графики и рассчитывает статистику.
    
    Args:
        dict_stats (Dict[str, Dict[str, float]]): Словарь со спектральными данными для обработки.
            Ключи - идентификаторы образцов, значения - словари с ключами 'B2', 'B3', 'B4', 'B5', 'B8A' и др.
        df (pd.DataFrame): Входной DataFrame с истинными значениями (столбец 'CHL').
        article (str): Идентификатор алгоритма для обработки. Допустимые значения:
            "beck", "molkov", "li", "makwinja", "boldanova", "karimi", "oreilly"
    
    Returns:
        Dict[str, List[Union[float, int, Dict, Any]]]: Словарь, где ключи - названия алгоритмов, а значения - списки со статистиками:
            [R², BIAS_log, BIAS, MAE_log, MAE, RMSE, STD, Sen_slope, MAPE, count, in_situ_dict, calc_dict]
            Подробности см. в докстринге calculate_statistics_and_draw_graphic
    
    Raises:
        ValueError: Если указан неверный идентификатор алгоритма (article)
    """
    
    statistics = {}
    
    match article:
        case "beck":
            beck_ndci_chl, beck_flh_violet_chl, beck_2bda_chl, beck_3bda_chl = defaultdict(dict), defaultdict(dict), defaultdict(dict), defaultdict(dict)
            
            for k, v in dict_stats.items():
                beck_ndci_chl[k], beck_flh_violet_chl[k], beck_2bda_chl[k], beck_3bda_chl[k] = beck(v)
            
            statistics.update(calculate_statistics_and_draw_graphic(df, 
                beck_ndci=beck_ndci_chl, 
                beck_flh_violet=beck_flh_violet_chl,
                beck_2bda=beck_2bda_chl, 
                beck_3bda=beck_3bda_chl))
        
        case "molkov":
            molkov_ndci, molkov_2bda, molkov_3bda, molkov_ph = {}, {}, {}, {}
            molkov_2b_linear_result, molkov_2b_poly_result = {}, {}
            molkov_ndci_linear_result, molkov_ndci_poly_result = {}, {}
            
            for k, v in dict_stats.items():
                molkov_ndci[k], molkov_2bda[k], molkov_3bda[k], molkov_ph[k], \
                molkov_2b_linear_result[k], molkov_2b_poly_result[k], \
                molkov_ndci_linear_result[k], molkov_ndci_poly_result[k] = molkov(v)
            
            molkov_ndci_list = convert_molkov_dict(molkov_ndci)
            molkov_2bda_list = convert_molkov_dict(molkov_2bda)
            molkov_3bda_list = convert_molkov_dict(molkov_3bda)
            molkov_ph_list = convert_molkov_dict(molkov_ph)
            
            statistics.update(calculate_statistics_and_draw_graphic(df,
                molkov_ndci_linear=molkov_ndci_list[0], molkov_ndci_poly=molkov_ndci_list[1],
                molkov_ndci_exp=molkov_ndci_list[2], molkov_ndci_power2=molkov_ndci_list[4],
                
                molkov_2b_linear=molkov_2bda_list[0], molkov_2b_poly=molkov_2bda_list[1],
                molkov_2b_exp=molkov_2bda_list[2], molkov_2b_power1=molkov_2bda_list[3],
                molkov_2b_power2=molkov_2bda_list[4],
                
                molkov_3b_linear=molkov_3bda_list[0], molkov_3b_poly=molkov_3bda_list[1],
                molkov_3b_exp=molkov_3bda_list[2], molkov_3b_power2=molkov_3bda_list[4],
                
                molkov_ph_linear=molkov_ph_list[0], molkov_ph_poly=molkov_ph_list[1],
                molkov_ph_exp=molkov_ph_list[2], molkov_ph_power1=molkov_ph_list[3],
                molkov_ph_power2=molkov_ph_list[4],
                
                molkov_2b_linear_result=molkov_2b_linear_result, 
                molkov_2b_poly_result=molkov_2b_poly_result,
                molkov_ndci_linear_result=molkov_ndci_linear_result, 
                molkov_ndci_poly_result=molkov_ndci_poly_result))
        
        case "li":
            li_candidate11, li_candidate12, li_candidate13 = {}, {}, {}
            li_candidate211, li_candidate212, li_candidate213, li_candidate22 = {}, {}, {}, {}
            
            for k, v in dict_stats.items():
                li_candidate11[k], li_candidate12[k], li_candidate13[k], li_candidate211[k], li_candidate212[k], \
                li_candidate213[k], li_candidate22[k] = li_candidate(v)
                print(k, li_candidate13[k], df.loc[k, 'CHL'])
    
            statistics.update(calculate_statistics_and_draw_graphic(df,
                li11=li_candidate11, li12=li_candidate12,
                li13=li_candidate13, li211=li_candidate211,
                li212=li_candidate212, li213=li_candidate213,
                li22=li_candidate22))
        
        case "makwinja":
            makwinja_ndci = {}
            
            for k, v in dict_stats.items():
                makwinja_ndci[k] = makwinja(v)
            
            statistics.update(calculate_statistics_and_draw_graphic(df, 
                makwinja_ndci=makwinja_ndci))
        
        case "boldanova":
            boldanova_b3 = {}
            for k, v in dict_stats.items():
                boldanova_b3[k] = boldanova(v)
            
            statistics.update(calculate_statistics_and_draw_graphic(df, 
                boldanova_b3=boldanova_b3))
        
        case "karimi":
            karimi_2bda, karimi_ndci, karimi_3bda = {}, {}, {}
            for k, v in dict_stats.items():
                karimi_2bda[k], karimi_ndci[k], karimi_3bda[k] = karimi(v)
            
            statistics.update(calculate_statistics_and_draw_graphic(df, 
                karimi_2bda=karimi_2bda, karimi_ndci=karimi_ndci,
                karimi_3bda=karimi_3bda))
        
        case "oreilly":
            oreilly_oc3 = {}
            for k, v in dict_stats.items():
                oreilly_oc3[k] = oreilly(v)
            
            statistics.update(calculate_statistics_and_draw_graphic(df, 
                oc3_msi=oreilly_oc3))
        
        case _:
            raise ValueError(f"Неизвестный алгоритм: {article}. "
                           "Допустимые значения: beck, molkov, li, makwinja, boldanova, karimi, oreilly")
    
    return statistics

In [ ]:
def convert_stat_dict_to_list_stat_and_values(arg_dict):
    """
    Args:
        arg_dict: {"name_algo1": [r2, bias, mae, n, {id: in situ}, {id: calculated}], "name_algo2": ...}
    
    Return: [{statustics (4 keys)}, {"name_algo" : [{id: in situ}, {id: calculated}]}]
    """
    
    statistics_dict = {}
    data_dict = {}
    for algo_name, algo_list in arg_dict.items():
        statistics_dict[algo_name] = algo_list[:-2]
        data_dict[algo_name] = [algo_list[-2], algo_list[-1]]
    return [statistics_dict, data_dict]

In [ ]:
df_all = pd.read_csv("../data/processed/chl_data.csv", index_col=0)

In [ ]:
df_rrs = pd.read_csv("../data/processed/rrs_data.csv", index_col=0)
dict_stats = df_rrs.T.to_dict()

# Results

### graphics

#### Beck (USA, 2016)

In [ ]:
beck_data = main(dict_stats, df_all, "beck")

#### Molkov (Russia, 2019)

In [ ]:
molkov_data = main(dict_stats, df_all, "molkov")

#### Li (China, 2021)

In [ ]:
li_candidate_data = main(dict_stats, df_all, "li")

#### Makwinja (Africa-Japan, 2022)

In [ ]:
makwinja_data = main(dict_stats, df_all, "makwinja")

#### Boldanova (Russia, 2022)

In [ ]:
boldanova_data = main(dict_stats, df_all, "boldanova")

#### Karimi (Iran, 2024)

In [ ]:
karimi_data = main(dict_stats, df_all, "karimi")

#### O'Reilly (USA, 2019)

In [ ]:
oreilly_data = main(dict_stats, df_all, "oreilly")

### statistics

In [ ]:
columns = ['R^2', 'Bias_log', 'Bias', 'MAE_log', 'MAE',	'RMSE', 'std', 'SS', 'MAPE', 'N']

#### Beck

In [ ]:
beck_stat = pd.DataFrame.from_dict(convert_stat_dict_to_list_stat_and_values(beck_data)[0], orient='index', columns=columns)
beck_stat

#### Molkov

In [ ]:
molkov_stat = pd.DataFrame.from_dict(convert_stat_dict_to_list_stat_and_values(molkov_data)[0], orient='index', columns=columns)
molkov_stat

#### Li

In [ ]:
li_candidate_stat = pd.DataFrame.from_dict(convert_stat_dict_to_list_stat_and_values(li_candidate_data)[0], orient='index', columns=columns)
li_candidate_stat

#### Makwinja 

In [ ]:
makwinja_stat = pd.DataFrame.from_dict(convert_stat_dict_to_list_stat_and_values(makwinja_data)[0], orient='index', columns=columns)
makwinja_stat

#### Boldanova

In [ ]:
boldanova_stat = pd.DataFrame.from_dict(convert_stat_dict_to_list_stat_and_values(boldanova_data)[0], orient='index', columns=columns)
boldanova_stat

#### Karimi

In [ ]:
karimi_stat = pd.DataFrame.from_dict(convert_stat_dict_to_list_stat_and_values(karimi_data)[0], orient='index', columns=columns)
karimi_stat

#### O'Reilly

In [ ]:
oreilly_stat = pd.DataFrame.from_dict(convert_stat_dict_to_list_stat_and_values(oreilly_data)[0], orient='index', columns=columns)
oreilly_stat

# Processing outliers

In [ ]:
def outliers(df_init: pd.DataFrame, threshold=3, n_sigma=1.1, window_size=3, **kwargs) -> Tuple[pd.DataFrame, Dict[str, List[Any]], Dict[str, List[int]]]:
    """
    Обнаруживает выбросы в данных с использованием метода Хампеля и обновляет DataFrame с флагами выбросов.
    
    Args:
        df_init (pd.DataFrame): Исходный DataFrame с данными для анализа
        threshold (float, optional): Пороговое значение для начального удаления явных выбросов. Defaults to 3.
        n_sigma (float, optional): Количество сигм для определения выбросов в методе Хампеля. Defaults to 1.1.
        window_size (int, optional): Размер скользящего окна для метода Хампеля. Defaults to 3.
        **kwargs: Словари с результатами алгоритмов в формате {имя_алгоритма: [статистики, истинные_значения, предсказанные_значения]}
    
    Returns:
        Tuple[pd.DataFrame, Dict[str, List[Any]], Dict[str, List[int]]]: Кортеж из трех элементов:
            1. df_pred: Обновленный DataFrame с добавленными колонками:
               - Предсказанные значения от алгоритмов
               - Флаги выбросов (flag_hampel_{алгоритм})
            2. stat: Словарь со статистиками по алгоритмам после удаления выбросов
            3. indecies_dict: Словарь с индексами обнаруженных выбросов для каждого алгоритма
    
    Raises:
        KeyError: Если в kwargs переданы некорректные данные без истинных/предсказанных значений
        ValueError: Если входные данные имеют некорректный формат
    """
    
    df_pred = copy.deepcopy(df_init)
    stat = dict()
    indecies_dict = dict()
    
    for algo_name, algo_dict in kwargs.items():
        try:
            in_situ = pd.Series(algo_dict[-2])
            predicted = pd.Series(algo_dict[-1])
        except (IndexError, TypeError):
            raise KeyError(f"Некорректные данные для алгоритма {algo_name}. Требуются истинные и предсказанные значения")
        
        residual = np.abs(in_situ - predicted)
        indecies_true = np.array([])
        
        if len((inds_array := np.where(residual > threshold)[0])):
            indecies_true = list(map(lambda t: in_situ.index[t], inds_array))
            residual = np.delete(residual, inds_array)
        
        result = hampel(residual, window_size=window_size, n_sigma=n_sigma)
        
        indecies = list(np.hstack((
            list(map(lambda t: in_situ.index[t], result.outlier_indices)),
            indecies_true
        )).astype(int))
        
        indecies_dict[algo_name] = indecies
        for i in indecies:
            if i in in_situ.index:
                in_situ.pop(i)
                predicted.pop(i)
        
        indecies_series = pd.Series({k: True if k in indecies else np.nan for k in indecies})
        indecies_series.name = f'flag_hampel_{algo_name}'
        
        predicted_in_table = pd.Series(algo_dict[-1])
        predicted_in_table.name = algo_name
        
        df_pred = pd.concat([df_pred, predicted_in_table, indecies_series], axis=1)
        
        in_situ_values = copy.deepcopy(in_situ).to_list()
        calc_chl = predicted.to_list()
        
        if len(calc_chl) > 3:
            stat[algo_name] = [*calculate_statistics(in_situ_values, calc_chl), 
                              len(calc_chl), 
                              in_situ, 
                              algo_dict]
            draw_graphic(in_situ_values, calc_chl, algo_name, save_folder="regional")
        else:
            print(f"Недостаточно данных для алгоритма {algo_name} (количество точек = {len(calc_chl)})")
    
    for i in df_pred.index:
        if df_pred.loc[i, 'datetime':].isna().sum() == len(kwargs) * 2:
            df_pred.drop(labels=[i], inplace=True)
    
    return df_pred, stat, indecies_dict

In [ ]:
clear_all_data_best = outliers(df_all, threshold=2,
                          li13_clear=li_candidate_data['li13'],
                          
                          oc3_msi_clear=oreilly_data['oc3_msi'])

In [ ]:
pd.DataFrame.from_dict({k: v[:-2] for k, v in clear_all_data_best[1].items()}, orient='index', columns=columns)